In [62]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

In [63]:
aoq1 = pd.read_csv(r'D:\Seksatta\MKG\summary lizard R&D\AOQ-VIS 010120-311223\aoq1.csv')
aoq2 = pd.read_csv(r'D:\Seksatta\MKG\summary lizard R&D\AOQ-VIS 010120-311223\aoq2.csv')
aoq3 = pd.read_csv(r'D:\Seksatta\MKG\summary lizard R&D\AOQ-VIS 010120-311223\aoq3.csv')

In [64]:
df = pd.concat([aoq1, aoq2, aoq3], keys=[1,2,3])

In [65]:
df.reset_index(inplace=True)

In [66]:
df.drop(columns = ['level_0', 'level_1'], inplace =True)


In [67]:
df.columns

Index(['SITE', 'FAC_MODEL', 'FAC_SERIAL', 'SYMPTOM_DESC',
       'FAILURE_PART_DESC_FINAL', 'FY_OF_CLAIM'],
      dtype='object')

In [68]:
df.drop_duplicates(subset = ['FAC_MODEL', 'FAC_SERIAL'], inplace=True)

In [69]:

df['SYMPTOM_DESC']=df['SYMPTOM_DESC'].str.replace(r'\b[dD]oest', 'Does', regex= True)
df['SYMPTOM_DESC']=df['SYMPTOM_DESC'].str.replace(r' /', '/', regex= True)
df['SYMPTOM_DESC']=df['SYMPTOM_DESC'].str.replace(r' /', '/', regex= True).str.rstrip('.')
df['SYMPTOM_DESC']=df['SYMPTOM_DESC'].str.replace(r'A/C', '', regex= True)
df['SYMPTOM_DESC']=df['SYMPTOM_DESC'].str.strip()
df['FAC_MODEL']=df['FAC_MODEL'].str.strip()
df['SYMPTOM_DESC']=df['SYMPTOM_DESC'].str.upper()
df['SYMPTOM_DESC']=df['SYMPTOM_DESC'].str.replace(r'10 :  DOES NOT OPERATE', '10 : DOES NOT OPERATE', regex= False)

In [70]:
df = df[df['FAILURE_PART_DESC_FINAL'].str.contains(r'\bPCB\b', case=False, regex=True)]

# Mapping with model and model group

In [71]:
model = pd.read_csv(r'D:\Seksatta\EQG\dit_info\model2.csv')

In [72]:
df = pd.merge(df, model[['base_model','model_name','group','i_o']], left_on = 'FAC_MODEL', right_on = 'model_name', how='left')
df.drop(columns=['model_name'], inplace=True, axis= 1)

In [73]:
df = df.dropna(subset=['group'])

In [74]:
df

,SITE,FAC_MODEL,FAC_SERIAL,SYMPTOM_DESC,FAILURE_PART_DESC_FINAL,FY_OF_CLAIM,base_model,group,i_o
0,DID,RZF100CYM4,E000655,00 : DOES NOT COOL/HEAT,738 : Inverter PCB [Electronic components],2021,BML-C,SKY,outdoor
1,DID,RZF100CYM4,E000665,00 : DOES NOT COOL/HEAT,738 : Inverter PCB [Electronic components],2021,BML-C,SKY,outdoor
2,DID,RZF100CVM4,E000563,10 : DOES NOT OPERATE,722 : Control PCB [Electronic components],2021,BML-C,SKY,outdoor
3,DID,RZF100CYM4,E000714,N/A : N/A,722 : Control PCB [Electronic components],2021,BML-C,SKY,outdoor
4,DID,RZF100CVM4,E000164,00 : DOES NOT COOL/HEAT,722 : Control PCB [Electronic components],2021,BML-C,SKY,outdoor
...,...,...,...,...,...,...,...,...,...
67984,SDS,RZF36CV2S,E011989,11 : ABNORMAL OPERATE,738 : Inverter PCB [Electronic components],2021,BML-C,SKY,outdoor
67985,SDS,RKD18GV2S,E018224,00 : DOES NOT COOL/HEAT,738 : Inverter PCB [Electronic components],2021,GBS,RA,outdoor
67986,DID,RKQ20SVM4,E007885,99 : OTHER,722 : Control PCB [Electronic components],2021,SMS,RA,outdoor
67987,SDS,RKQ12SV2S,E022750,11 : ABNORMAL OPERATE,738 : Inverter PCB [Electronic components],2021,GSI,RA,outdoor


In [84]:
df.columns

Index(['SITE', 'FAC_MODEL', 'FAC_SERIAL', 'SYMPTOM_DESC',
       'FAILURE_PART_DESC_FINAL', 'FY_OF_CLAIM', 'base_model', 'group', 'i_o'],
      dtype='object')

In [91]:
df1 = df.groupby(['FY_OF_CLAIM', 'group', 'i_o', 'SYMPTOM_DESC', 'FAILURE_PART_DESC_FINAL'])['SITE'].size().reset_index()

In [93]:
df1.to_excel(r'pcb_service.xlsx')